In [ ]:
using FrameFun, BasisFunctions, Plots, FastTransforms, PGFPlotsX

In [ ]:
fun = x->exp(cos(6eltype(x)(pi)*x))
Ns = 3:2:61
function frame_plot(Ns,errors,norms)
   scatter(Ns,errors .+ eps();title="Fout",yscale=:log10,layout=(1,2),
        size=(500,200),subplot=1,legend=false,ylims=(1e-16,1e2),background_color=:transparent,foreground_color=:black)
    scatter!(Ns,norms .+ eps();subplot=2,yscale=:log10,legend=false,ylims=(1e-1,1e6),title="Coefficientengrootte") 
end
function frame_plotpgf(Ns,errors,norms)
    @pgf GroupPlot({group_style = {group_size = "2 by 1",},legend=false,width=150,height=100,},
        {title="Fout",ymode="log",ymin=1e-16,ymax=1e2,xmin=0,xmax=Ns[end]+1}, 
        PlotInc({},Table([Ns,errors .+ eps()])),
        {title="Coefficientengrootte",ymode="log",ymin=1e-1,ymax=1e6,xmin=0,xmax=Ns[end]+1}, 
        PlotInc({},Table([Ns,norms .+ eps()]))  
    )
end

In [ ]:
p = platform(extensionframe(Fourier{BigFloat}(1),big(0.0)..big(0.5)))
norms1 = zeros(BigFloat,length(Ns))
errors1 = zeros(BigFloat,length(Ns))
for (i,N) in enumerate(Ns)
    opts = (;L=2N,verbose=false,solverstyle=DirectStyle(),directsolver=:qr,samplingstyle=GramStyle())
    F = Fun(fun,p,N;opts...)
    g = sampling_grid(p,N;L=3N)
    norms1[i],errors1[i] = norm(coefficients(F)),norm(F(g)-fun.(g),Inf)
end
frame_plot(Ns,errors1,norms1)

In [ ]:
p = platform(extensionframe(Fourier(1),(0.0)..(0.5)))
norms2 = zeros(length(Ns))
errors2 = zeros(length(Ns))
for (i,N) in enumerate(Ns)
    opts = (;L=2N,verbose=false,solverstyle=DirectStyle(),directsolver=:qr,samplingstyle=GramStyle())
    F = Fun(fun,p,N;opts...)
    g = sampling_grid(p,N;L=3N)
    norms2[i],errors2[i] = norm(coefficients(F)),norm(F(g)-fun.(g),Inf)
end
frame_plot(Ns,errors2,norms2)

In [ ]:
norms3 = zeros(length(Ns))
errors3 = zeros(length(Ns))
for (i,N) in enumerate(Ns)
    opts = (;solverstyle=AZStyle())
    F = Fun(fun,p,N;opts...)
    g = sampling_grid(p,N;L=3N)
    norms3[i],errors3[i] = norm(coefficients(F)),norm(F(g)-fun.(g),Inf)
end
frame_plot(Ns,errors3,norms3)

In [ ]:
frame_plot(Ns,errors1,norms1)
savefig("oversampling1")

In [ ]:
frame_plot(Ns,errors2,norms2)
savefig("oversampling2")

In [ ]:
frame_plot(Ns,errors3,norms3)
savefig("oversampling3")

In [ ]:
pgfsave("oversampling1.tex",frame_plotpgf(Ns,errors1,norms1),include_preamble=false)
pgfsave("oversampling2.tex",frame_plotpgf(Ns,errors2,norms2),include_preamble=false)
pgfsave("oversampling3.tex",frame_plotpgf(Ns,errors3,norms3),include_preamble=false)